In [60]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#A code to run ordinary least squares with associated statistics
#Jeremy Kedziora
#24 March 2016
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

#import libraries
import warnings
import numpy as np    #import for arrays
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.learning_curve import learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.ensemble import ExtraTreesClassifier
from matplotlib.colors import ListedColormap
from scipy.optimize import minimize    #import for optimization
%matplotlib inline

In [2]:
#define functions
def maker(N,n_vars,kind = 'linear'):
    """A function to generate Monte Carlo linear regression data"""
    x = []    #an empty list to hold the data
    y = np.zeros(N)    #an array to hold the dependent variable
    b = []    #an empty list to hold the true bs
    i = 1
    while i <= n_vars:    #loop over the variables we want to create
        x_i = np.random.normal(loc = 0.0, scale = 1.0, size = N)    #generate the data
        x.append(x_i)    #add it to the list of data
        b_i = np.random.normal(loc = 0.0, scale = 1.0)    #draw a random effect for this variable
        b.append(b_i)    #add it to the list of effects
        y = y + b_i*x_i    #add the variable effect to the dependent variable
        i += 1    #index up i
    
    x.append(np.ones(N))    #and a column of ones for a constant
    b_i = np.random.normal(loc = 0.0, scale = 1.0)    #draw a random intercept
    b.append(b_i)    #append this intercept to the effects
    if kind == 'linear':
        y = b_i + y + np.random.normal(loc = 0.0, scale = 1.0, size = N)    #add the normally distributed error term and the intercept
    if kind == 'logit':
        y = (np.random.uniform(0,1,len(y)) < np.exp(b_i + y)/(1 + np.exp(b_i + y)))*1
    return [np.array(x).T,np.array(y),np.array(b)]

In [3]:
def OLS_mle(b,X,y):
    """A function to compute OLS coefficients using MLE"""
    s2 = 1.0#math.exp(b[len(b) - 1])    #exponentiate the variance to ensure that it is positive
    xb = X.dot(b)    #compute the means
    return -1*sum(-0.5*np.log(s2) - (y - xb)**2/(2*s2))    #return the log likelihood

In [4]:
def logit_mle(b,X,y):
    """A function to compute logit coefficients using MLE"""
    xb = X.dot(b)    #compute the means
    return -1*sum(y*xb - np.log(1+np.exp(xb)))    #return the log likelihood

In [5]:
Data = maker(100000,3,kind = 'logit')    #make logit data
X = Data[0]    #pull out explanatory variables
y = Data[1]    #pull out dependent variable
b = Data[2]    #pull out true coefficients

b = np.random.uniform(0,1,4)*0.01    #set starting values
Coefficients = minimize(logit_mle, x0 = b, args = (X,y), method = 'BFGS').x    #maximize the log-likelihood
print(Coefficients)    #print out the coefficients
print(Data[2])    #print out the true betas

[-0.56515772 -0.12758514 -0.00702179  0.08378708]
[-0.57259197 -0.11743945 -0.00440701  0.08862368]


In [6]:
Data = maker(100000,3,kind = 'logit')    #make logit data
X = Data[0]    #pull out explanatory variables
y = Data[1]    #pull out dependent variable
b = Data[2]    #pull out true coefficients

b = np.random.uniform(0,1,4)*0.01    #set starting values
Coefficients = minimize(logit_mle, x0 = b, args = (X,y), method = 'BFGS').x    #maximize the log-likelihood
print(Coefficients)    #print out the coefficients
print(Data[2])    #print out the true betas

[-0.4767515  -0.2237589   0.19042283 -0.44034815]
[-0.47207773 -0.2331351   0.18882702 -0.44045565]


1. Use the uploaded MLE_Examples_1 notebook to generate fake logistic data.
2. Generate data sets with 10, 20, 30, 40, 50, 60, 70, 80, 90, and 100 features.
3. Split each data set into a training set and a test set.
4. Use sklearn to apply a decision tree and a random forest model to each training set. 
5. And then use the trained model to predict in the test set.
6. Finally, assess the percent correctly predicted by each model in the test set.
7. What happens to the percent correctly predicted as the number of features increases?

### 1. Use the uploaded MLE_Examples_1 notebook to generate fake logistic data.
### 2. Generate data sets with 10, 20, 30, 40, 50, 60, 70, 80, 90, and 100 features.

In [15]:
data10 = maker(10000,10,kind = 'logit')
data20 = maker(10000,20,kind = 'logit')
data30 = maker(10000,30,kind = 'logit')
data40 = maker(10000,40,kind = 'logit')
data50 = maker(10000,50,kind = 'logit')
data60 = maker(10000,60,kind = 'logit')
data70 = maker(10000,70,kind = 'logit')
data80 = maker(10000,80,kind = 'logit')
data90 = maker(10000,90,kind = 'logit')
data100 = maker(10000,100,kind = 'logit')

In [16]:
y10 = data10[1]
x10 = data10[0]

### 3. Split each data set into a training set and a test set.

In [25]:
X10_train, X10_test, y10_train, y10_test = train_test_split(x10, y10, test_size=0.3, stratify=y10)

### 4. Use sklearn to apply a decision tree and a random forest model to each training set. 

In [29]:
forest = RandomForestClassifier()
forest.fit(X10_train, y10_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### 5. And then use the trained model to predict in the test set.

In [45]:
y10_pred = forest.predict(X10_test)

### 6. Finally, assess the percent correctly predicted by each model in the test set.

In [46]:
print("accuracy: ", accuracy_score(y10_test, y10_pred))
print("precision: ", precision_score(y10_test, y10_pred))
print("recall: ", recall_score(y10_test, y10_pred))
print("f1 score: ", f1_score(y10_test, y10_pred))

accuracy:  0.801333333333
precision:  0.843525179856
recall:  0.807692307692
f1 score:  0.825219941349


### 7. What happens to the percent correctly predicted as the number of features increases?

In [65]:
datasets = [data10, data20, data30, data40, data50, data60, data70, data80, data90, data100]
warnings.filterwarnings('ignore')

for i in datasets:
    X = i[0]
    y = i[1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
    forest = RandomForestClassifier()
    forest.fit(X, y)
    y_pred = forest.predict(X_test)
    print("accuracy", str(datasets.index(i)+1) + ":", accuracy_score(y_test, y_pred))
#     print("precision", str(datasets.index(i)+1) + ":", precision_score(y_test, y_pred))
#     print("recall", str(datasets.index(i)+1) + ":", recall_score(y_test, y_pred))
#     print("f1 score", str(datasets.index(i)+1) + ":", f1_score(y_test, y_pred))
#     print("\n")

accuracy 1: 0.993333333333
accuracy 2: 0.994333333333
accuracy 3: 0.993
accuracy 4: 0.990333333333
accuracy 5: 0.990333333333
accuracy 6: 0.992
accuracy 7: 0.99
accuracy 8: 0.991666666667
accuracy 9: 0.995333333333
accuracy 10: 0.992666666667
